## Ingredientize ASA24 food records using FNDDS ingredients

In [1]:
#import packages
import numpy as np
import pandas as pd

In [2]:
#Load data
fndds = pd.read_csv('data/fndds/fndds_2018/fndds2018.csv')
fndds.head()

,Food code,Main food description,WWEIA Category number,WWEIA Category description,Seq num,Ingredient code,Ingredient description,Ingredient weight (g),Retention code,Moisture change (%)
0,11000000,"Milk, human",9602,Human milk,1,1107,"Milk, human, mature, fluid",100.0,0,0.0
1,11100000,"Milk, NFS",1004,"Milk, reduced fat",1,1077,"Milk, whole, 3.25% milkfat, with added vitamin D",35.0,0,0.0
2,11100000,"Milk, NFS",1004,"Milk, reduced fat",2,1079,"Milk, reduced fat, fluid, 2% milkfat, with add...",37.0,0,0.0
3,11100000,"Milk, NFS",1004,"Milk, reduced fat",3,1082,"Milk, lowfat, fluid, 1% milkfat, with added vi...",17.0,0,0.0
4,11100000,"Milk, NFS",1004,"Milk, reduced fat",4,1085,"Milk, nonfat, fluid, with added vitamin A and ...",11.0,0,0.0


In [3]:
fndds.dtypes

Food code                       int64
Main food description          object
WWEIA Category number           int64
WWEIA Category description     object
Seq num                         int64
Ingredient code                 int64
Ingredient description         object
Ingredient weight (g)         float64
Retention code                  int64
Moisture change (%)           float64
dtype: object

In [4]:
#Select columns: dropping WWEIA cols
fndds.drop(columns=['WWEIA Category number', 'WWEIA Category description'], inplace=True)

#Rename column 'Food code' to 'FoodCode' for merging with ASA data
fndds.columns=['FoodCode',
               'fndds_description',
               'Seq num',
               'Ingredient code',
               'Ingredient description',
               'Ingredient weight (g)',
               'Retention code',
               'Moisture change (%)']

In [5]:
asa24 = pd.read_csv('data/asa24/fl100_recalls_qcd.csv')
print(asa24.shape)
asa24.head()

(19373, 103)


,UserName,UserID,RecallNo,RecallAttempt,RecallStatus,IntakeStartDateTime,IntakeEndDateTime,ReportingDate,Lang,Occ_No,...,F_TOTAL,F_CITMLB,F_OTHER,D_TOTAL,D_MILK,D_YOGURT,D_CHEESE,FoodComp,Food_Description,Modified
0,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,MultiGrain Cheerios,NaN
1,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Water, tap",NaN
2,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Frankfurter or hot dog, beef",NaN
3,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Roll, white, soft",NaN
4,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Mustard,NaN


In [6]:
asa24['FoodCode'].nunique()

2143

In [7]:
asa24['Food_Description'].nunique()

2285

In [8]:
asa24 = asa24[asa24['FoodCode'] != 9]

In [9]:
# Rename column to distinguish ASA food description from FNDDS food description before merging datasets
asa24.rename(columns={'Food_Description': 'asa_description'}, inplace=True)

In [10]:
# Merge ASA/FNDDS datasets to ingredientize the ASA24 food records. 
# This will create an ingredient for each mixed food from foodcodes
asa_fndds = pd.merge(asa24, fndds, on='FoodCode')
asa_fndds.head()

,UserName,UserID,RecallNo,RecallAttempt,RecallStatus,IntakeStartDateTime,IntakeEndDateTime,ReportingDate,Lang,Occ_No,...,FoodComp,asa_description,Modified,fndds_description,Seq num,Ingredient code,Ingredient description,Ingredient weight (g),Retention code,Moisture change (%)
0,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,1,...,1.0,MultiGrain Cheerios,NaN,Cereal (General Mills Cheerios Multigrain),1,8087,"Cereals ready-to-eat, GENERAL MILLS, Multi-Gra...",100.0,0,0.0
1,5043,81a66438-6cae-4acf-a4af-552908ab4619,4,0,1,11/4/15 0:00,11/5/15 0:00,11/5/15,1,1,...,1.0,MultiGrain Cheerios,NaN,Cereal (General Mills Cheerios Multigrain),1,8087,"Cereals ready-to-eat, GENERAL MILLS, Multi-Gra...",100.0,0,0.0
2,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,1,...,1.0,"Water, tap",NaN,"Water, tap",1,14411,"Beverages, water, tap, drinking",100.0,0,0.0
3,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,2,...,1.0,"Water, tap",NaN,"Water, tap",1,14411,"Beverages, water, tap, drinking",100.0,0,0.0
4,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,3,...,1.0,"Water, tap",NaN,"Water, tap",1,14411,"Beverages, water, tap, drinking",100.0,0,0.0


In [11]:
#Rearrange columns of interest
cols_to_order = ['UserName',
 'FoodCode',
 'asa_description',
 'fndds_description',
 'Seq num',
 'Ingredient code',
 'Ingredient description']
new_columns = cols_to_order + (asa_fndds.columns.drop(cols_to_order).tolist())
asa_fndds = asa_fndds[new_columns]
asa_fndds.head()

,UserName,FoodCode,asa_description,fndds_description,Seq num,Ingredient code,Ingredient description,UserID,RecallNo,RecallAttempt,...,F_OTHER,D_TOTAL,D_MILK,D_YOGURT,D_CHEESE,FoodComp,Modified,Ingredient weight (g),Retention code,Moisture change (%)
0,5001,57308400.0,MultiGrain Cheerios,Cereal (General Mills Cheerios Multigrain),1,8087,"Cereals ready-to-eat, GENERAL MILLS, Multi-Gra...",6fbb2168-fd69-4842-896b-56b4735cf652,2,0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,100.0,0,0.0
1,5043,57308400.0,MultiGrain Cheerios,Cereal (General Mills Cheerios Multigrain),1,8087,"Cereals ready-to-eat, GENERAL MILLS, Multi-Gra...",81a66438-6cae-4acf-a4af-552908ab4619,4,0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,100.0,0,0.0
2,5001,94000100.0,"Water, tap","Water, tap",1,14411,"Beverages, water, tap, drinking",6fbb2168-fd69-4842-896b-56b4735cf652,2,0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,100.0,0,0.0
3,5001,94000100.0,"Water, tap","Water, tap",1,14411,"Beverages, water, tap, drinking",6fbb2168-fd69-4842-896b-56b4735cf652,2,0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,100.0,0,0.0
4,5001,94000100.0,"Water, tap","Water, tap",1,14411,"Beverages, water, tap, drinking",6fbb2168-fd69-4842-896b-56b4735cf652,2,0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,100.0,0,0.0


In [12]:
asa_set = set(asa24['asa_description'].unique())
asa_fndds_set = set(asa_fndds['asa_description'].unique())

In [13]:
food_desc_diff = asa_set.difference(asa_fndds_set)
print(len(food_desc_diff), 'foodcodes not mapping from ASA24 to FNDDS2018')

483 foodcodes not mapping from ASA24 to FNDDS2018


In [14]:
food_desc_diff = pd.DataFrame(food_desc_diff)
food_desc_diff.columns = ['missing_food_descriptions']

In [15]:
food_list = food_desc_diff['missing_food_descriptions'].tolist()
food_list

['Almonds, dry roasted, without salt',
 'Chicken, wing, coated, baked or fried, prepared with skin, skin/coating eaten W/ VEGETABLE OIL, NFS (INCLUDE OIL, NFS)',
 'Vegetable combination (including carrots, broccoli, and/or dark-green leafy), cooked, no sauce, NS as to fat added in cooking',
 'Corn, cooked, NS as to form, NS as to color, NS as to fat added in cooking',
 'Grits, cooked, corn or hominy, regular, fat not added in cooking W/ MILK',
 'Cheese, natural, Cheddar or American type',
 'Bread stick, soft, prepared with garlic and parmesan cheese',
 'Oatmeal, cooked, regular, NS as to fat added in cooking',
 'Chicken, thigh, coated, baked or fried, prepared skinless, coating eaten',
 'Black, brown, or Bayo beans, dry, cooked, NS as to fat added in cooking',
 'White rice with tomato sauce W/ VEGETABLE OIL, NFS (INCLUDE OIL, NFS)',
 'Spaghetti with tomato sauce and vegetables',
 'Rice, white, cooked, regular, fat added in cooking W/ BUTTER, NFS',
 'Sweet potato, fried',
 'Vegetable co

In [16]:
missing_foods = asa24['asa_description'].isin(food_list)
fndds_missing = asa24[missing_foods]
fndds_missing = fndds_missing.drop_duplicates(subset = ['asa_description'])
fndds_missing

,UserName,UserID,RecallNo,RecallAttempt,RecallStatus,IntakeStartDateTime,IntakeEndDateTime,ReportingDate,Lang,Occ_No,...,F_TOTAL,F_CITMLB,F_OTHER,D_TOTAL,D_MILK,D_YOGURT,D_CHEESE,FoodComp,asa_description,Modified
14,5001,6fbb2168-fd69-4842-896b-56b4735cf652,2,0,7,5/22/15 0:00,5/23/15 0:00,5/23/15,1,5,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Onion, young green, cooked, from fresh, NS as ...",NaN
24,5002,ebbaa09a-f313-4c4f-a44e-4c4c33d83af1,2,0,1,12/20/15 0:00,12/21/15 0:00,12/21/15,1,2,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Ground beef, less than 80% lean, cooked (forme...",NaN
32,5002,ebbaa09a-f313-4c4f-a44e-4c4c33d83af1,2,0,1,12/20/15 0:00,12/21/15 0:00,12/21/15,1,3,...,0.124968,0.000000,0.124968,0.000000,0.0000,0.0,0.000000,1.0,"Fig, dried, cooked, unsweetened",NaN
47,5005,59ec1000-a384-4f54-823c-b35659d5e8f0,2,0,7,7/6/15 0:00,7/7/15 0:00,7/7/15,1,2,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Peppers, hot, cooked, NS as to form, fat added...",NaN
76,5005,59ec1000-a384-4f54-823c-b35659d5e8f0,2,0,7,7/6/15 0:00,7/7/15 0:00,7/7/15,1,4,...,0.000000,0.000000,0.000000,0.172464,0.0000,0.0,0.172464,1.0,"Pizza with meat, prepared from frozen, thin crust",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18911,9038,f37113fd-a8ea-4892-b7e5-311da3622712,4,0,2,5/14/19 0:00,5/15/19 0:00,5/15/19,1,4,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Carrots, cooked, NS as to form, glazed",NaN
18962,9040,cf050b28-95f3-4920-b285-3ad2e85b63b5,4,0,2,4/30/19 0:00,5/1/19 0:00,5/1/19,1,1,...,1.166625,1.166625,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Strawberries, frozen, with sugar",NaN
19089,9046,a43b9578-b4e8-483a-8fef-ba51f0083a2e,4,0,2,5/19/19 0:00,5/20/19 0:00,5/20/19,1,5,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Cauliflower, cooked, NS as to form, NS as to f...",NaN
19095,9047,b7e96cce-d3e2-46cc-b697-7d523de73b64,4,0,2,6/22/19 0:00,6/23/19 0:00,6/23/19,1,1,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,1.0,"Beans, string, yellow, cooked, from fresh, fat...",NaN


In [17]:
fndds_missing_foods = fndds_missing[['FoodCode', 'asa_description']]
fndds_missing_foods.to_csv('data/fndds/fndds_remap_code_2011_2018/fndds_missing_foods.csv', index=None, header=True)